In [1]:
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime as dt
from datetime import timedelta
import datetime
import json
import os
from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
# import datetime
import random
# from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
# from py_vollib.black_scholes.greeks import analytical as greeks
# import time
# import ks_api_client
# from kiteconnect import KiteConnect
# from kiteconnect import KiteTicker

In [24]:
#Slice data
all_df = pd.read_csv("data_df/Data 2022-01-29 19_52_56 team.csv")
all_df['current_datetime'] = pd.to_datetime(all_df['current_datetime'])
all_df['expiry_datetime'] = pd.to_datetime(all_df['expiry_datetime'])
# all_df.rename(columns={'candle_5_0_close_last':'candle_5_0_close',
#                         'candle_5_0_open_last':'candle_5_0_open',
#                         'candle_5_5_close_last':'candle_5_5_close',
#                         'candle_5_5_open_last':'candle_5_0_open'},
#                         inplace=True)

from_time = dt(2022,1,29,19,52,55)
till_time = dt(2023,1,29,15,50,1)

slice = all_df[(all_df['current_datetime']>=from_time) &\
   (all_df['current_datetime']<=till_time)].head(5)

slice

,underlying_name,current_datetime,current_pnl,strategy_pnl,brokerage_pnl,max_pnl,trailing_pnl,current_ltp,expiry_datetime,current_position,...,is_hedged,is_closed,candle_t_2_start,candle_t_2_end,candle_t_2_open,candle_t_2_close,candle_t_1_start,candle_t_1_end,candle_t_1_open,candle_t_1_close
0,NIFTY,2022-01-29 19:52:56.255361,0.00,0.0,0.00,0.00,0.00,17101.95,2022-02-03 15:30:00,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NIFTY,2022-01-29 19:52:57.903455,5224.98,5225.0,-0.02,5224.98,0.00,17101.95,2022-02-03 15:30:00,NaN,...,True,False,2022-01-29 19:50:00,2022-01-29 19:51:00,NaN,NaN,2022-01-29 19:51:00,2022-01-29 19:52:00,NaN,NaN
2,NIFTY,2022-01-29 19:52:59.620937,-216200.04,-216200.0,-0.04,5224.98,-221425.02,17101.95,2022-02-03 15:30:00,strangle,...,True,False,2022-01-29 19:50:00,2022-01-29 19:51:00,NaN,NaN,2022-01-29 19:51:00,2022-01-29 19:52:00,NaN,NaN
3,NIFTY,2022-01-29 19:53:01.479992,-0.08,0.0,-0.08,5224.98,-5225.06,17101.95,2022-02-03 15:30:00,NaN,...,False,True,2022-01-29 19:51:00,2022-01-29 19:52:00,NaN,NaN,2022-01-29 19:52:00,2022-01-29 19:53:00,17101.95,17101.95
4,NIFTY,2022-01-29 19:53:02.294278,-0.08,0.0,-0.08,5224.98,-5225.06,17101.95,2022-02-03 15:30:00,NaN,...,False,True,2022-01-29 19:51:00,2022-01-29 19:52:00,NaN,NaN,2022-01-29 19:52:00,2022-01-29 19:53:00,17101.95,17101.95


In [41]:
def print_all (*args, **kwargs):
    print("args")
    for i in args:
        print(i)
    print("kwargs")
    for k, v in kwargs.items():
        print(k,v)
def keep_log (**kwargs_decorator):
    default_return = None
    if 'default_return' in kwargs_decorator:
        default_return = kwargs_decorator.pop('default_return')
    def decorator_function (original_function):
        def wrapper_function(*args,**kwargs):
            #Capruting className and functionName
            class_function_name_dict = {\
                    'className': args[0].__class__.__name__,
                    'functionName': original_function.__name__}
            print(class_function_name_dict)
            print(args[0].__class__.__name__)
            print_all(**kwargs,**{'args':args[1:]},**kwargs_decorator)
            return original_function(*args,**kwargs)
        return wrapper_function
    return decorator_function

@keep_log(another=2)
def addnos (a,b, mul = 1):
    return (a+b)*mul

In [42]:
addnos(2,3, mul=2)

{'className': 'int', 'functionName': 'addnos'}
int
args
kwargs
mul 2
args (3,)
another 2


10

SyntaxError: invalid syntax (2889402527.py, line 2)

In [1]:
#Data Streaming using Websocket
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime as dt
from datetime import timedelta
import datetime
# import datetime
import random
from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
from py_vollib.black_scholes.greeks import analytical as greeks
import time
import ks_api_client
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

tokens = []
def set_kite (kite_api_key=None, kite_access_token=None):

    kite = KiteConnect(api_key=kite_api_key)
    kite.set_access_token(kite_access_token)
    kws = KiteTicker(kite_api_key,kite_access_token)
    return kite, kws


def get_instruments_book (kite):
    nse = pd.DataFrame(kite.instruments("NSE"))
    nfo = pd.DataFrame(kite.instruments("NFO"))
    kite_instruments_book = nfo.append(nse)
    kite_instruments_book['expiry'] =  pd.to_datetime(kite_instruments_book['expiry'], 
                                            format='%Y-%m-%d') 
    kite_instruments_book['expiry_datetime'] = kite_instruments_book['expiry'] + \
                                        timedelta (hours=15, minutes =30)
    return kite_instruments_book


def tokenLookup(instrument_df,symbol_list):
    """Looks up instrument token for a given script from instrument dump"""
    token_list = []
    for symbol in symbol_list:
        token_list.append(int(instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]))
    return token_list


def get_fno_instrument_id (instruments_df,strike, underlying, call_put, expiry_datetime) -> str:
        
            
    instrument_id = str(instruments_df[(instruments_df['name']==underlying)
            &(instruments_df['instrument_type']==call_put)
            &(instruments_df['expiry_datetime']==expiry_datetime)
            &(instruments_df['strike']==strike)]['instrument_token'].iloc[0])
    
    if not isinstance(instrument_id,str):
        instrument_id = str(instrument_id.iloc[0])

    return instrument_id


def get_multiple_fno_instrument_id (instruments_df, fno_df) -> pd.Series:
        # fno_df should have underlying, call_put, expiry_datetime and strike column

    instrument_id = fno_df.merge(instruments_df,how='left',\
            left_on=['underlying','call_put','expiry_datetime','strike'], 
            right_on=['name','instrument_type','expiry_datetime','strike'])\
            ['instrument_token'].astype(int)

    return instrument_id

def on_close(ws, code, reason):
    print("closed connection on close: {} {}".format(code, reason))


def on_error(ws, code, reason):
    print("closed connection on error: {} {}".format(code, reason))


def on_noreconnect(ws):
    print("Reconnecting the websocket failed")


def on_reconnect(ws, attempt_count):
    print("Reconnecting the websocket: {}".format(attempt_count))


def on_order_update(ws, data):
    print("order update: ", data)


def on_connect(ws,response):
    global tokens
    print("on connect: {}".format(response))
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    #print("on connect: {}".format(response))
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL,tokens) # Set all token tick in `full` mode.
    #ws.set_mode(ws.MODE_FULL,[tokens[0]])  # Set one token tick in `full` mode.

counter = 0
combined_df = pd.DataFrame()
def on_ticks(ws,ticks):
    # Callback to receive ticks.
    # print("Ticks: {}".format(ticks))
    # print(ticks)
    # populate_df.delay(ticks)
    global counter
    global combined_df
    counter +=1
    tick_df = pd.DataFrame(ticks)[['instrument_token',\
        'last_price','volume','buy_quantity',\
        'sell_quantity','oi','timestamp','depth']]
    combined_df = combined_df.append(tick_df)
    # if dt.now().second == 0:
    #     combined_df.to_csv("tick_stream.csv",sep="|")
    #     print(f"saved,{counter}",end='\r')
    # else:
    print(f"         {counter}",end='\r')

def populate_df (ticks):
    global counter
    global combined_df
    counter +=1
    tick_df = pd.DataFrame(ticks)[['instrument_token',\
        'last_price','volume','buy_quantity',\
        'sell_quantity','oi','timestamp','depth']]
    combined_df = combined_df.append(tick_df)
    # if dt.now().second == 0:
    #     combined_df.to_csv("tick_stream.csv",sep="|")
    #     print(f"saved,{counter}",end='\r')
    # else:
    print(f"         {counter}",end='\r')


kite, kws = set_kite(kite_api_key="5ytarhiur9g1jebq",
    kite_access_token="oAwZpfJqmnb33FKt5wEczWCZmSdaTMKs")
instruments_df = get_instruments_book(kite)

next_expiry_datetime = instruments_df[\
                                    (instruments_df['name']=='NIFTY') 
                                    & (instruments_df['instrument_type'] == 'CE')]\
                                    ['expiry_datetime'].min()


# names = ['NIFTY']
names = ['NIFTY 50', 'INDIA VIX','NIFTY BANK', 'NIFTY','BANKNIFTY']
# names = ['NIFTY 50', 'INDIA VIX','NIFTY BANK']

tokens = list(instruments_df[(instruments_df['name']\
        .isin(names))&(instruments_df['expiry_datetime']==next_expiry_datetime)]\
            ['instrument_token'].astype(int))
print(type(tokens))
print(len(tokens))
# strike = 18_000.0
# underlying = 'NIFTY'
# expiry_datetime=dt(2022,1,27,15,30)
# fno_df = pd.DataFrame({'strike':[strike,strike,strike],\
#                         'underlying':[underlying,underlying,underlying],
#                         'expiry_datetime':[expiry_datetime,expiry_datetime,expiry_datetime],
#                         'call_put':['CE','PE','PE']})
# fno_df['tokens'] = get_multiple_fno_instrument_id(
#         instruments_df=instruments_df,
#         fno_df=fno_df)
# tokens = list(fno_df['tokens'])
# print(tokens)


<class 'list'>
464


In [2]:
# Assign the callbacks.
combined_df = pd.DataFrame()
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close
kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

# kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
# kws.connect(disable_ssl_verification=True)
kws.connect()

on connect: {"peer": "tcp4:13.127.118.127:443", "headers": {"date": "Mon, 24 Jan 2022 09:29:33 GMT", "connection": "upgrade", "upgrade": "websocket", "sec-websocket-accept": "h6FNntMirrG/aXf+PLjdol4KHBI="}, "version": 18, "protocol": null, "extensions": []}


Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


closed connection on error: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
closed connection on close: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


'/Users/hg/Space/Code/AlgoTrader/broker_secret.json'